In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
RANDOM_SEED = 42

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
df = pd.read_csv("./kidney-stone-dataset.csv")
df = df.drop(columns=["Unnamed: 0"])
X = df[["ph", "osmo", "urea", "calc", "gravity", "cond"]]
y = df["target"]  # or correct target column nameprint(df[:5])
X = df[["ph", "osmo", "urea", "calc", "gravity", "cond"]]
X = torch.from_numpy(np.array(X)).type(dtype=torch.float32)
y = torch.from_numpy(np.array(y)).type(dtype=torch.float32)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED
)


class Kidney_Stone(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(in_features=6, out_features=24)
        self.layer2 = nn.Linear(in_features=24, out_features=12)
        self.layer3 = nn.Linear(in_features=12, out_features=1)

    def forward(self, X):
        X = self.layer1(X)
        X = self.layer2(X)
        X = self.layer3(X)
        return X


torch.manual_seed(RANDOM_SEED)
model_7 = Kidney_Stone()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X = torch.tensor(X_scaled, dtype=torch.float32)

In [ ]:

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(params=model_7.parameters(), lr=0.01)

In [ ]:
print(X_train.size())
print(len(model_7.state_dict()))
epochs = 200
train_losses = []
test_losses = []
for epoch in range(epochs):
    model_7.train()
    y_logits = model_7(X_train).squeeze()
    loss = loss_fn(y_logits, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model_7.eval()
    with torch.inference_mode():
        test_logits = model_7(X_test).squeeze()
        test_loss = loss_fn(test_logits, y_test)

        train_preds = torch.round(torch.sigmoid(y_logits))
        test_preds = torch.round(torch.sigmoid(test_logits))
    if epoch % 10 == 0:
        train_losses.append(loss.item())
        test_losses.append(test_loss.item())
        print(
            f"Epoch {epoch} | Train Loss: {loss.item():.4f} | Test Loss: {test_loss.item():.4f}"
        )

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(train_losses, label="Train Loss")
plt.plot(test_losses, label="Test Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Kidney Stone Model Training")
plt.legend()
plt.grid(True)
plt.show()